# RETURN → APPROVAL 변환 작업
activities 컬럼에서 RETURN 타입을 APPROVAL로 변환하고, actionComment에 [반려] 표시 추가

## 0단계: DB 연결 설정

In [1]:
import pymysql
import json
from pprint import pprint

# DB 연결 설정
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='1234',
    database='any_approval',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

print("DB 연결 성공!")

DB 연결 성공!


## 1단계: 현황 파악

In [2]:
# RETURN이 포함된 레코드 조회
cursor = conn.cursor()

cursor.execute("SELECT id, activities FROM documents WHERE activities LIKE '%RETURN%'")
rows_with_return = cursor.fetchall()

print(f"=== 현황 파악 ===")
print(f"RETURN이 포함된 레코드 수: {len(rows_with_return)}개")

# RETURN 항목 총 건수 계산
total_return_count = 0
for row in rows_with_return:
    activities = json.loads(row['activities']) if isinstance(row['activities'], str) else row['activities']
    for act in activities:
        if act.get('type') == 'RETURN' or act.get('actionLogType') == 'RETURN':
            total_return_count += 1

print(f"RETURN 항목 총 건수: {total_return_count}건")

=== 현황 파악 ===
RETURN이 포함된 레코드 수: 38개
RETURN 항목 총 건수: 48건


## 2단계: 데이터 정합성 검증

In [3]:
# type과 actionLogType 일치 여부 검증
normal_count = 0
type_only_return = []  # type만 RETURN
actionLogType_only_return = []  # actionLogType만 RETURN

for row in rows_with_return:
    activities = json.loads(row['activities']) if isinstance(row['activities'], str) else row['activities']
    for idx, act in enumerate(activities):
        t = act.get('type')
        alt = act.get('actionLogType')
        
        if t == 'RETURN' and alt == 'RETURN':
            normal_count += 1
        elif t == 'RETURN' and alt != 'RETURN':
            type_only_return.append({
                'doc_id': row['id'],
                'index': idx,
                'type': t,
                'actionLogType': alt,
                'name': act.get('name')
            })
        elif t != 'RETURN' and alt == 'RETURN':
            actionLogType_only_return.append({
                'doc_id': row['id'],
                'index': idx,
                'type': t,
                'actionLogType': alt,
                'name': act.get('name')
            })

print(f"=== 데이터 정합성 검증 ===")
print(f"✓ 정상 (type=RETURN, actionLogType=RETURN): {normal_count}건")
print(f"⚠ type만 RETURN: {len(type_only_return)}건")
print(f"⚠ actionLogType만 RETURN: {len(actionLogType_only_return)}건")

if type_only_return:
    print(f"\n[type만 RETURN인 데이터]")
    for item in type_only_return[:10]:  # 최대 10개만 표시
        print(f"  doc_id={item['doc_id']}, name={item['name']}, type={item['type']}, actionLogType={item['actionLogType']}")

if actionLogType_only_return:
    print(f"\n[actionLogType만 RETURN인 데이터]")
    for item in actionLogType_only_return[:10]:  # 최대 10개만 표시
        print(f"  doc_id={item['doc_id']}, name={item['name']}, type={item['type']}, actionLogType={item['actionLogType']}")

# 검증 결과 요약
if len(type_only_return) == 0 and len(actionLogType_only_return) == 0:
    print(f"\n✅ 모든 RETURN 데이터가 정상입니다. 다음 단계로 진행 가능!")
else:
    print(f"\n⚠️ 이상 데이터가 있습니다. 확인 후 처리 방법을 결정하세요.")

=== 데이터 정합성 검증 ===
✓ 정상 (type=RETURN, actionLogType=RETURN): 48건
⚠ type만 RETURN: 0건
⚠ actionLogType만 RETURN: 0건

✅ 모든 RETURN 데이터가 정상입니다. 다음 단계로 진행 가능!


## 3단계: 변환 미리보기 (Dry Run)

In [4]:
def transform_activities(activities_json):
    """
    RETURN을 APPROVAL로 변환하고 actionComment에 [반려] 추가
    변환된 JSON과 변환 건수를 반환
    """
    activities = json.loads(activities_json) if isinstance(activities_json, str) else activities_json
    changed_count = 0
    
    for act in activities:
        # type 또는 actionLogType 중 하나라도 RETURN이면 변환
        if act.get('type') == 'RETURN' or act.get('actionLogType') == 'RETURN':
            # actionComment 앞에 [반려] 추가
            original_comment = act.get('actionComment', '') or ''
            if not original_comment.startswith('[반려]'):
                act['actionComment'] = '[반려]' + original_comment
            
            # type과 actionLogType 모두 APPROVAL로 변경
            act['type'] = 'APPROVAL'
            act['actionLogType'] = 'APPROVAL'
            changed_count += 1
    
    return json.dumps(activities, ensure_ascii=False), changed_count

In [5]:
# 미리보기: 샘플 5개 출력
print("=== 변환 미리보기 (Dry Run) ===\n")

preview_count = min(5, len(rows_with_return))
total_preview_changes = 0

for i, row in enumerate(rows_with_return[:preview_count]):
    print(f"--- 문서 ID: {row['id']} ---")
    
    original_activities = json.loads(row['activities']) if isinstance(row['activities'], str) else row['activities']
    transformed_json, change_count = transform_activities(row['activities'])
    transformed_activities = json.loads(transformed_json)
    
    # RETURN이었던 항목만 비교 출력
    for j, (orig, trans) in enumerate(zip(original_activities, transformed_activities)):
        if orig.get('type') == 'RETURN' or orig.get('actionLogType') == 'RETURN':
            print(f"\n  [항목 {j+1}] {orig.get('name')}")
            print(f"  변경 전: type={orig.get('type')}, actionLogType={orig.get('actionLogType')}")
            print(f"           actionComment=\"{orig.get('actionComment', '')}\"")
            print(f"  변경 후: type={trans.get('type')}, actionLogType={trans.get('actionLogType')}")
            print(f"           actionComment=\"{trans.get('actionComment', '')}\"")
    
    total_preview_changes += change_count
    print()

print(f"=== 미리보기 요약 ===")
print(f"전체 대상 레코드: {len(rows_with_return)}개")
print(f"전체 변환 예정 항목: {total_return_count}건")

=== 변환 미리보기 (Dry Run) ===

--- 문서 ID: 3681 ---

  [항목 3] 김민서
  변경 전: type=RETURN, actionLogType=RETURN
           actionComment="반려합니다."
  변경 후: type=APPROVAL, actionLogType=APPROVAL
           actionComment="[반려]반려합니다."

--- 문서 ID: 5419 ---

  [항목 3] 김민서
  변경 전: type=RETURN, actionLogType=RETURN
           actionComment="임현준위원님으로 정정하여 올려주세요"
  변경 후: type=APPROVAL, actionLogType=APPROVAL
           actionComment="[반려]임현준위원님으로 정정하여 올려주세요"

--- 문서 ID: 7641 ---

  [항목 3] 김민서
  변경 전: type=RETURN, actionLogType=RETURN
           actionComment="법인카드 품의에서 제외 해주세요"
  변경 후: type=APPROVAL, actionLogType=APPROVAL
           actionComment="[반려]법인카드 품의에서 제외 해주세요"

  [항목 4] 정주연
  변경 전: type=RETURN, actionLogType=RETURN
           actionComment="2025년 7월 17일에
최기원 책임이 메일로 공유한 양식을 사용해주세요.


1. 결재선 : 팀장 승인 → 본부장 승인 → 회계담당 합의 → 경영기획팀장 승인

2. 정산긴간 대상 : 매월 1일~말일 사용내역

3. 정산방법 : 당월 사용내역은 차월 경영기획팀에서 5일까지 발송 후 10일까지 그룹웨어에 정산하여 보고


당월 사용내역은 차월 경영기획팀에서 5일까지 발송 후 10일까지 그룹웨어에 정산하여 보고"
  변경 후: type=APPROVAL, acti

## 4단계: 실제 적용
⚠️ 이 셀을 실행하면 실제 DB가 변경됩니다!

In [7]:
# 실제 적용 여부 확인
CONFIRM_APPLY = True  # True로 변경하면 실제 적용됨

if not CONFIRM_APPLY:
    print("⚠️ CONFIRM_APPLY를 True로 변경해야 실제 적용됩니다.")
else:
    print("=== 실제 적용 시작 ===")
    
    cursor = conn.cursor()
    updated_records = 0
    updated_items = 0
    
    for row in rows_with_return:
        transformed_json, change_count = transform_activities(row['activities'])
        
        if change_count > 0:
            cursor.execute(
                "UPDATE documents SET activities = %s WHERE id = %s",
                (transformed_json, row['id'])
            )
            updated_records += 1
            updated_items += change_count
    
    conn.commit()
    
    print(f"✅ 적용 완료!")
    print(f"   변경된 레코드: {updated_records}개")
    print(f"   변경된 항목: {updated_items}건")

=== 실제 적용 시작 ===
✅ 적용 완료!
   변경된 레코드: 38개
   변경된 항목: 48건


## 5단계: 적용 후 검증

In [8]:
# 적용 후 검증 (4단계 실행 후에 실행하세요)
cursor = conn.cursor()

# RETURN이 남아있는지 확인
cursor.execute("SELECT COUNT(*) as cnt FROM documents WHERE activities LIKE '%\"RETURN\"%'")
remaining_return = cursor.fetchone()['cnt']

# [반려]가 붙은 항목 수 확인
cursor.execute("SELECT id, activities FROM documents WHERE activities LIKE '%[반려]%'")
rows_with_banlye = cursor.fetchall()

banlye_count = 0
for row in rows_with_banlye:
    activities = json.loads(row['activities']) if isinstance(row['activities'], str) else row['activities']
    for act in activities:
        if '[반려]' in (act.get('actionComment') or ''):
            banlye_count += 1

print(f"=== 적용 후 검증 ===")
print(f"남아있는 RETURN 레코드: {remaining_return}개")
print(f"[반려] 표시된 항목 수: {banlye_count}건")
print(f"원래 RETURN 항목 수: {total_return_count}건")

if remaining_return == 0 and banlye_count == total_return_count:
    print(f"\n✅ 검증 완료! 모든 변환이 정상적으로 적용되었습니다.")
else:
    if remaining_return > 0:
        print(f"\n⚠️ RETURN이 아직 {remaining_return}개 남아있습니다.")
    if banlye_count != total_return_count:
        print(f"\n⚠️ [반려] 개수가 일치하지 않습니다. (예상: {total_return_count}, 실제: {banlye_count})")

=== 적용 후 검증 ===
남아있는 RETURN 레코드: 0개
[반려] 표시된 항목 수: 48건
원래 RETURN 항목 수: 48건

✅ 검증 완료! 모든 변환이 정상적으로 적용되었습니다.


## 연결 종료

In [9]:
cursor.close()
conn.close()
print("DB 연결 종료")

DB 연결 종료
